In [58]:
%pip install langchain langchain_openai


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


##  Initializing LLMs

In [59]:
from langchain_openai import ChatOpenAI


In [60]:
#OpenAI
import os 

openai_api_key = os.environ.get("OPENAI_API_KEY")
openaiLLm = ChatOpenAI(api_key=openai_api_key,model_name="gpt-4o", temperature=0)

In [61]:
#Ollama
# Make sure Ollama is running locally e.g `ollama run llama3.2`  https://github.com/ollama/ollama

ollamallm = ChatOpenAI(api_key="-Nothing to set with Ollama", model="llama3.2:latest",base_url="http://localhost:11434/v1")

## Asking about EOL support date for JDK23

In [62]:
# Input prompt
from langchain_core.messages import HumanMessage
messages = [HumanMessage("What is the end support life date for JDK version 23.0.2")]

In [63]:
## Closed World LLM output - Ollama - (Local offline)

aimsg1 = ollamallm.invoke(messages)
print(aimsg1.content)

I don't have the most up-to-date information on the end support life dates for JDK versions.

However, I can suggest some options to help you find the answer:

1. Check the official Oracle website: You can visit the Oracle website and search for the JDK version 23.0.2 to see if they have any information on its end-of-life date.
2. Check the OpenJDK website: Since OpenJDK is a community-driven project, their website may have more up-to-date information on the end support life dates for different JDK versions.
3. Contact Oracle Support: If you need to find the specific end support life date for JDK version 23.0.2, I recommend contacting Oracle Support directly. They should be able to provide you with the most recent and accurate information.

I apologize for not being able to provide a more definitive answer. If you have any further questions or concerns, feel free to ask!


In [64]:
## Closed World LLM output - OpenAI - (Online)

aimsg2 = openaiLLm.invoke(messages)
print(aimsg2.content)

As of my last update, JDK 23 has not been released, so there is no specific end of support life date for JDK version 23.0.2. Oracle typically provides updates and support for each Long-Term Support (LTS) release for a period of at least eight years, with the first three years being Premier Support and the remaining five years being Extended Support. Non-LTS releases, on the other hand, generally receive updates for a shorter period, often until the next feature release is available.

For the most accurate and up-to-date information, you should check Oracle's official Java SE Support Roadmap or the documentation provided by the OpenJDK community.


## Introducing Tools With ORacle Java Release API

Create a tool relying on an external API provided by Oracle to track java releases 
https://docs.oracle.com/en-us/iaas/jms/doc/public-api-oracle-java-releases.html


In [77]:
import requests
from langchain_core.tools import tool

@tool
def getOracleJavaEolSupportDate(releaseVersion: str) -> str:
    """ Get the Oracle JDK releases info : EOL Support dates , CPU, LTS status, Licence ... """
    api_url =f"https://java.oraclecloud.com/javaReleases"
    response = requests.get(api_url)
    if response.status_code ==200:
        data = response.json()
        print(f"data:{data}")
        releases= data["items"]
        customerReleases = [
            r for r in releases
             if ((r.get("jdkDetails", {}).get("releaseVersion") == releaseVersion)
             or (r.get("jdkDetails", {}).get("jdkVersion") == releaseVersion)
             or (releaseVersion.startswith(r.get("jdkDetails", {}).get("jdkVersion"))))
        ]       
        print((f"customerReleases:{customerReleases}"))
        if len(customerReleases)<1:
            raise ValueError(f"No version {releaseVersion} found ")
            return
    
        item= customerReleases[0]
        print(f"Item:{item} ")
        return item.get("jdkDetails").get("endOfSupportLifeDate")
    else:
        raise ValueError(f"Faile to fetch EOL data for {releaseVersion}")


In [67]:
getOracleJavaEolSupportDate("21")

data:{'items': [{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True, 'latestReleaseVersion': '23.0.2', 'isLts': False}, 'licenseDetails': {'displayName': 'Oracle No-Fee Terms and Conditions', 'licenseType': 'NFTC', 'licenseUrl': 'https://java.com/freeuselicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/23-0-2-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '23.0.2', 'releaseFullVersion': 'jdk-23.0.2-58+7', 'securityStatus': 'UP_TO_DATE'}, {'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/en/java/javase/21', 'endOfSupportLifeDate': '2031-09-19T23:59:59.000Z', 'jdkVersion': '21', 'isSupportedVersion': Tr

'2031-09-19T23:59:59.000Z'

In [78]:
getOracleJavaEolSupportDate("23.0.2")

data:{'items': [{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True, 'latestReleaseVersion': '23.0.2', 'isLts': False}, 'licenseDetails': {'displayName': 'Oracle No-Fee Terms and Conditions', 'licenseType': 'NFTC', 'licenseUrl': 'https://java.com/freeuselicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/23-0-2-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '23.0.2', 'releaseFullVersion': 'jdk-23.0.2-58+7', 'securityStatus': 'UP_TO_DATE'}, {'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/en/java/javase/21', 'endOfSupportLifeDate': '2031-09-19T23:59:59.000Z', 'jdkVersion': '21', 'isSupportedVersion': Tr

'2025-03-17T23:59:59.000Z'

In [34]:
getOracleJavaEolSupportDate("1.8.0_431")

data:{'items': [{'artifactContentTypes': ['JDK', 'JRE', 'SERVER_JRE'], 'daysUnderSecurityBaseline': 9, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/8', 'endOfSupportLifeDate': '2030-12-31T23:59:59.000Z', 'jdkVersion': '8', 'isSupportedVersion': True, 'latestReleaseVersion': '1.8.0_441', 'isLts': True}, 'licenseDetails': {'displayName': 'Oracle Technology Network', 'licenseType': 'OTN', 'licenseUrl': 'https://java.com/otnlicense'}, 'releaseDate': '2024-10-15T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/8u431-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '1.8.0_431', 'releaseFullVersion': 'jdk-1.8.0_431-b10', 'securityStatus': 'UPDATE_REQUIRED'}]}
Item:{'artifactContentTypes': ['JDK', 'JRE', 'SERVER_JRE'], 'daysUnderSecurityBaseline': 9, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/8', 'endOfSupportLifeDate': '2030-12-31T23:59:59.000Z', 'jdkVersio

'2030-12-31T23:59:59.000Z'

In [35]:
getOracleJavaEolSupportDate("1.8")

data:{'items': []}


ValueError: No version 1.8 found 

In [ ]:
getOracleJavaEolSupportDate("1.8.0_441")

data:{'items': [{'artifactContentTypes': ['JDK', 'JRE', 'SERVER_JRE'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/8', 'endOfSupportLifeDate': '2030-12-31T23:59:59.000Z', 'jdkVersion': '8', 'isSupportedVersion': True, 'latestReleaseVersion': '1.8.0_441', 'isLts': True}, 'licenseDetails': {'displayName': 'Oracle Technology Network', 'licenseType': 'OTN', 'licenseUrl': 'https://java.com/otnlicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/8u441-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '1.8.0_441', 'releaseFullVersion': 'jdk-1.8.0_441-b07', 'securityStatus': 'UP_TO_DATE'}]}
Item:{'artifactContentTypes': ['JDK', 'JRE', 'SERVER_JRE'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/8', 'endOfSupportLifeDate': '2030-12-31T23:59:59.000Z', 'jdkVersion': '

'2030-12-31T23:59:59.000Z'

## Plugin Tools to our LLMs - (in progress)

In [12]:
tooling= [getEolSupportDate]
openaiLLmWithTools = openaiLLm.bind_tools([getOracleJavaEolSupportDate])
ollamallmWithTools = ollamallm.bind_tools([getOracleJavaEolSupportDate])



In [13]:
ai_msg = openaiLLmWithTools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
   tool_msg = getOracleJavaEolSupportDate.invoke(tool_call)
  
   print(tool_msg.name)
   print(tool_call['args'])
   print(tool_msg.content)
   messages.append(tool_msg)
   print()
final_response = openaiLLmWithTools.invoke(messages)
print(final_response.content)

data:{'items': [{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True, 'latestReleaseVersion': '23.0.2', 'isLts': False}, 'licenseDetails': {'displayName': 'Oracle No-Fee Terms and Conditions', 'licenseType': 'NFTC', 'licenseUrl': 'https://java.com/freeuselicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/23-0-2-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '23.0.2', 'releaseFullVersion': 'jdk-23.0.2-58+7', 'securityStatus': 'UP_TO_DATE'}]}
Item:{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True

In [ ]:
# Augmented Ollama 
messages = [HumanMessage("What is the end support life date for JDK version 23.0.2")]
ai_msg = ollamallmWithTools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
   tool_msg = getOracleJavaEolSupportDate.invoke(tool_call)
  
   print(tool_msg.name)
   print(tool_call['args'])
   print(tool_msg.content)
   messages.append(tool_msg)
   print()
final_response = ollamallmWithTools.invoke(messages)
print(final_response.content)

data:{'items': [{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True, 'latestReleaseVersion': '23.0.2', 'isLts': False}, 'licenseDetails': {'displayName': 'Oracle No-Fee Terms and Conditions', 'licenseType': 'NFTC', 'licenseUrl': 'https://java.com/freeuselicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/23-0-2-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '23.0.2', 'releaseFullVersion': 'jdk-23.0.2-58+7', 'securityStatus': 'UP_TO_DATE'}]}
Item:{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True

## Using OpenAI Agent Executor Framework
OpenAI provides various pattern to run agents with functions and tools 
https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_tools/


In [20]:
# Imports
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent



In [21]:
# Create Agent 

prompt = hub.pull("hwchase17/openai-tools-agent")

prompt

/Users/enono/.pyenv/versions/3.12.6/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [24]:
tools=[getOracleJavaEolSupportDate]
agent = create_openai_tools_agent(openaiLLm,tools , prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [ ]:
agent_executor.invoke({"input": "What is the end support life date for JDK version 23.0.2?"})
agent_executor



> Entering new AgentExecutor chain...

Invoking: `getEolSupportDate` with `{'releaseVersion': '23.0.2'}`


data:{'items': [{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True, 'latestReleaseVersion': '23.0.2', 'isLts': False}, 'licenseDetails': {'displayName': 'Oracle No-Fee Terms and Conditions', 'licenseType': 'NFTC', 'licenseUrl': 'https://java.com/freeuselicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/23-0-2-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '23.0.2', 'releaseFullVersion': 'jdk-23.0.2-58+7', 'securityStatus': 'UP_TO_DATE'}]}
Item:{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/j

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [69]:
agent_executor.invoke({"input": "What is the end support life date for JDK version 1.8.0_431?"})





> Entering new AgentExecutor chain...

Invoking: `getEolSupportDate` with `{'releaseVersion': '1.8.0_431'}`


data:{'items': [{'artifactContentTypes': ['JDK', 'JRE', 'SERVER_JRE'], 'daysUnderSecurityBaseline': 10, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/8', 'endOfSupportLifeDate': '2030-12-31T23:59:59.000Z', 'jdkVersion': '8', 'isSupportedVersion': True, 'latestReleaseVersion': '1.8.0_441', 'isLts': True}, 'licenseDetails': {'displayName': 'Oracle Technology Network', 'licenseType': 'OTN', 'licenseUrl': 'https://java.com/otnlicense'}, 'releaseDate': '2024-10-15T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/8u431-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '1.8.0_431', 'releaseFullVersion': 'jdk-1.8.0_431-b10', 'securityStatus': 'UPDATE_REQUIRED'}]}
Item:{'artifactContentTypes': ['JDK', 'JRE', 'SERVER_JRE'], 'daysUnderSecurityBaseline': 10, 'isScriptDownloadAvailable': True, 'jdkDetai

{'input': 'What is the end support life date for JDK version 1.8.0_431?',
 'output': 'The end of support life date for JDK version 1.8.0_431 is December 31, 2030.'}

## Using LangGraph React Agents

In [71]:
from langgraph.prebuilt import create_react_agent

tools=[getOracleJavaEolSupportDate]
llm = ChatOpenAI(api_key=openai_api_key,model_name="gpt-4o", temperature=0)
agent =create_react_agent(llm,tools=tools )
query = " What is the ORacle EOL suppport date for java 23"
messages = agent.invoke({"messages": [("human", query)]})
{
    "prompt": query,
    "response": messages["messages"][-1].content,
}

data:{'items': [{'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/javase/23', 'endOfSupportLifeDate': '2025-03-17T23:59:59.000Z', 'jdkVersion': '23', 'isSupportedVersion': True, 'latestReleaseVersion': '23.0.2', 'isLts': False}, 'licenseDetails': {'displayName': 'Oracle No-Fee Terms and Conditions', 'licenseType': 'NFTC', 'licenseUrl': 'https://java.com/freeuselicense'}, 'releaseDate': '2025-01-21T00:00:00.000Z', 'releaseNotesUrl': 'https://www.oracle.com/java/technologies/javase/23-0-2-relnotes.html', 'releaseType': 'CPU', 'releaseVersion': '23.0.2', 'releaseFullVersion': 'jdk-23.0.2-58+7', 'securityStatus': 'UP_TO_DATE'}, {'artifactContentTypes': ['JDK'], 'daysUnderSecurityBaseline': 0, 'isScriptDownloadAvailable': True, 'jdkDetails': {'docUrl': 'https://docs.oracle.com/en/java/javase/21', 'endOfSupportLifeDate': '2031-09-19T23:59:59.000Z', 'jdkVersion': '21', 'isSupportedVersion': Tr

{'prompt': ' What is the ORacle EOL suppport date for java 23',
 'response': 'The Oracle EOL (End of Life) support date for Java 23 is March 17, 2025.'}